# CS441 – Graph Isomorphism Project
This notebook implements the four algorithms required in the project:
1. Brute-force isomorphism search  
2. Backtracking  
3. Degree-based backtracking  
4. Weisfeiler–Leman backtracking  

Each function strictly follows the specifications and returns:
- `(True, mapping)` if the graphs are isomorphic  
- `(False, [])` otherwise

Graphs are loaded from text files according to the required format.


## Section 1 — Imports
This section loads all required Python packages.

The project does not allow external libraries, so we only use:
- itertools (for permutations)
- typing (for type clarity)
- copy (for safe duplication of partitions/mappings)


In [ ]:
# ============================================
# Section 1 - Imports
# ============================================

from itertools import permutations
from typing import List, Set, Dict, Tuple, Union
from copy import deepcopy

## Section 2 — Graph Loading and Basic Utilities

This section contains helper functions used throughout the algorithms:

### `load_graph(path)`
Reads a graph from a text file and returns its adjacency structure.

### `check_isomorphism_from_mapping`
Given a bijection ρ between nodes of `G1` and `G2`, checks whether all edges
are preserved.

### `quick_invariants_match`
Performs fast necessary checks before running expensive algorithms:
- same number of nodes
- same degree multiset

### `_build_degree_partition`
Creates the initial partition of nodes based on degrees (Question 3).


In [ ]:
# ============================================
# Section 2 - Graph loading and basic helpers
# ============================================

from typing import List, Set, Dict, Tuple, Union

def load_graph(path: str) -> List[Set[int]]:
    """
    Load an undirected graph from a text file with the format:

      Line 1:  n  m
          n = number of nodes (0..n-1)
          m = number of edges (not strictly needed here)
      Line 2: neighbors of node 0 (space-separated)
      Line 3: neighbors of node 1
      ...
      Line n+1: neighbors of node n-1

    Returns:
      adj: list of length n where adj[i] is a set of neighbors of node i.
    """
    with open(path, "r") as f:
        first_line = f.readline().split()
        if not first_line:
            raise ValueError("Empty file or invalid format")

        n = int(first_line[0])
        # m = int(first_line[1])  # not needed for the algorithms

        adj = [set() for _ in range(n)]
        for i in range(n):
            line = f.readline()
            if not line:
                break
            neighbors = line.split()
            for nb in neighbors:
                v = int(nb)
                adj[i].add(v)
                # Input is assumed symmetric (undirected).
                # If it was not, we could force symmetry with: adj[v].add(i)
    return adj


def check_isomorphism_from_mapping(
    adj1: List[Set[int]],
    adj2: List[Set[int]],
     mapping: List[int]
) -> bool:
    """
    Check if 'mapping' is an isomorphism between adj1 and adj2.

    Condition:
      (i, j) in E1  <=>  (mapping[i], mapping[j]) in E2
    for all 0 <= i < j < n.
    """
    n = len(adj1)
    for i in range(n):
        mi = mapping[i]
        for j in range(i + 1, n):
            mj = mapping[j]
            edge1 = j in adj1[i]
            edge2 = mj in adj2[mi]
            if edge1 != edge2:
                return False
    return True


def quick_invariants_match(adj1: List[Set[int]], adj2: List[Set[int]]) -> bool:
    """
    Quick necessary checks before running expensive algorithms:

    - Same number of nodes.
    - Same multiset of degrees.
    """
    if len(adj1) != len(adj2):
        return False

    deg1 = sorted(len(adj1[i]) for i in range(len(adj1)))
    deg2 = sorted(len(adj2[i]) for i in range(len(adj2)))
    return deg1 == deg2


def _build_degree_partition(adj: List[Set[int]]) -> Tuple[List[int], List[List[int]]]:
    """
    Build the degree partition of a graph.

    IMPORTANT (project requirement):
    - The part index j IS the degree.
    - P(j) contains exactly the nodes with degree j.

    Returns:
      node_to_part: list of size n, node_to_part[v] = degree(v)
      parts      : list of lists, parts[d] = [nodes of degree d]
                    (some parts may be empty if no node has that degree)
    """
    n = len(adj)
    degrees = [len(adj[v]) for v in range(n)]
    if n == 0:
        return [], []

    max_deg = max(degrees)

    # parts[d] will contain all nodes of degree d
    parts: List[List[int]] = [[] for _ in range(max_deg + 1)]
    node_to_part = [-1] * n

    for v in range(n):
        d = degrees[v]
        parts[d].append(v)
        node_to_part[v] = d

    return node_to_part, parts


## Section 3 — Question 1: Brute Force Algorithm

This algorithm generates **all permutations** of nodes of the second graph
and checks which one preserves adjacency.

Although extremely slow for large graphs (because it tries `n!` mappings),
it is required as part of the project.

### Function provided:
- `brute_force(path1, path2)`


In [ ]:
# ============================================
# Section 3 - Question 1: Brute force
# ============================================

def brute_force(path1: str, path2: str) -> Tuple[bool, List[int]]:
    """
    Brute-force graph isomorphism (Question 1).

    Idea:
      - Generate all bijections ρ: V1 → V2 (all permutations of {0, ..., n-1}).
      - For each permutation, test the isomorphism condition.

    Returns:
      (True, mapping_list)  if an isomorphism exists,
      (False, [])           otherwise.
    """
    adj1 = load_graph(path1)
    adj2 = load_graph(path2)

    # Quick rejection before heavy search
    if not quick_invariants_match(adj1, adj2):
        print(False, [])
        return False, []

    n = len(adj1)

    for perm in permutations(range(n)):
        mapping = list(perm)
        if check_isomorphism_from_mapping(adj1, adj2, mapping):
            print(True, mapping)
            return True, mapping

    print(False, [])
    return False, []


## Section 4 — Question 2: Backtracking Algorithm

This algorithm builds the mapping incrementally:

1. Start with node `v1` in `G1`
2. Try assigning it to every unused node in `G2`
3. After each tentative mapping, check edge-consistency with previously mapped nodes
4. If consistent → continue
5. If not consistent → undo the assignment (**backtrack**) and try another choice

This drastically reduces the search space compared to brute force.

### Provided:
- `_backtracking_recursive()`
- `backtracking(path1, path2)`


In [ ]:
# ============================================
# Section 4 - Question 2: Backtracking
# ============================================

def _backtracking_recursive(
    idx: int,
    adj1: List[Set[int]],
    adj2: List[Set[int]],
    mapping: List[int],
    used: List[bool],
    order: List[int]
) -> bool:
    """
    Recursive helper for backtracking.

    Parameters:
      idx   : index in 'order' of the node we are mapping now.
      order : ordering of vertices in G1, usually [0, 1, ..., n-1].

    At step idx:
      - Let v1 = order[idx].
      - Try each unused vertex v2 in G2 as image.
      - Check edge consistency with already-mapped vertices.
      - If consistent, recurse; otherwise backtrack.
    """
    n = len(adj1)
    if idx == n:
        # All vertices are mapped.
        return True

    v1 = order[idx]

    for v2 in range(n):
        if used[v2]:
            continue

        # Tentative assignment v1 -> v2.
        mapping[v1] = v2

        # Check isomorphism condition for edges involving v1 and already mapped u1.
        consistent = True
        for u1 in range(n):
            if u1 == v1 or mapping[u1] == -1:
                continue
            u2 = mapping[u1]
            edge1 = (u1 in adj1[v1])
            edge2 = (u2 in adj2[v2])
            if edge1 != edge2:
                consistent = False
                break

        if not consistent:
            mapping[v1] = -1
            continue

        # Continue deeper.
        used[v2] = True
        if _backtracking_recursive(idx + 1, adj1, adj2, mapping, used, order):
            return True
        # Backtrack.
        used[v2] = False
        mapping[v1] = -1

    return False


def backtracking(path1: str, path2: str) -> Tuple[bool, List[int]]:
    """
    Greedy backtracking algorithm (Question 2).

    We map v1, v2, ..., vn one by one and test the isomorphism condition
    for the first i vertices at step i.

    Returns:
      (True, mapping_list)  if an isomorphism exists,
      (False, [])           otherwise.
    """
    adj1 = load_graph(path1)
    adj2 = load_graph(path2)

    if not quick_invariants_match(adj1, adj2):
        print(False, [])
        return False, []

    n = len(adj1)
    mapping = [-1] * n
    used = [False] * n
    order = list(range(n))   # could be changed to a heuristic order if desired.

    success = _backtracking_recursive(0, adj1, adj2, mapping, used, order)

    if success:
        print(True, mapping)
        return True, mapping
    else:
        print(False, [])
        return False, []


## Section 5 — Question 3: Backtracking with Degree Partitioning

This improves the previous algorithm:

A node in `G1` with degree `d` can only map to nodes in `G2` that also have degree `d`.

This reduces the candidate set drastically and speeds up the search.

### Provided:
- `_degree_backtracking_recursive()`
- `degree_backtracking(path1, path2)`
- alias: `backtracking_degree = degree_backtracking`


In [ ]:
# ============================================
# Section 5 - Question 3: Backtracking with degree partitioning
# ============================================

def _degree_backtracking_recursive(
    idx: int,
    adj1: List[Set[int]],
    adj2: List[Set[int]],
    mapping: List[int],
    used: List[bool],
    order: List[int],
    deg1: List[int],
    deg2_nodes: Dict[int, List[int]]
) -> bool:
    """
    Recursive helper for degree-constrained backtracking.

    Only tries candidates in G2 that have the same degree as the current
    node v1 in G1.
    """
    n = len(adj1)
    if idx == n:
        return True

    v1 = order[idx]
    d = deg1[v1]
    candidates = deg2_nodes.get(d, [])

    for v2 in candidates:
        if used[v2]:
            continue

        mapping[v1] = v2

        # Check consistency with already mapped nodes.
        consistent = True
        for u1 in range(n):
            if u1 == v1 or mapping[u1] == -1:
                continue
            u2 = mapping[u1]
            edge1 = (u1 in adj1[v1])
            edge2 = (u2 in adj2[v2])
            if edge1 != edge2:
                consistent = False
                break

        if not consistent:
            mapping[v1] = -1
            continue

        used[v2] = True
        if _degree_backtracking_recursive(
            idx + 1, adj1, adj2, mapping, used, order, deg1, deg2_nodes
        ):
            return True
        used[v2] = False
        mapping[v1] = -1

    return False


def degree_backtracking(path1: str, path2: str) -> Tuple[bool, List[int]]:
    """
    Backtracking with degree partitioning (Question 3).

    A node v in G1 with degree d can only map to nodes of degree d in G2.

    Returns:
      (True, mapping_list)  if isomorphic,
      (False, [])           otherwise.
    """
    adj1 = load_graph(path1)
    adj2 = load_graph(path2)

    if not quick_invariants_match(adj1, adj2):
        print(False, [])
        return False, []

    n = len(adj1)
    mapping = [-1] * n
    used = [False] * n
    order = list(range(n))

    deg1 = [len(adj1[v]) for v in range(n)]
    deg2 = [len(adj2[v]) for v in range(n)]

    # degree -> list of nodes in G2 with that degree
    deg2_nodes: Dict[int, List[int]] = {}
    for v in range(n):
        d = deg2[v]
        deg2_nodes.setdefault(d, []).append(v)

    success = _degree_backtracking_recursive(
        0, adj1, adj2, mapping, used, order, deg1, deg2_nodes
    )

    if success:
        print(True, mapping)
        return True, mapping
    else:
        print(False, [])
        return False, []


# Alias because the statement names this function backtracking_degree.
backtracking_degree = degree_backtracking


In [ ]:
def backtracking_degree(path1: str, path2: str):
    """
    Wrapper to match the required function name in the assignment.
    It simply calls degree_backtracking under the hood.
    """
    return degree_backtracking(path1, path2)

## Section 6 — Question 4: Stable Partitioning (Weisfeiler–Leman Refinement)

This section implements the classical 1-dimensional Weisfeiler–Leman refinement:

1. Start from the degree partition
2. Repeatedly refine parts:
   - nodes remain in the same part only if they have the same multiset
     of neighbor-part labels
3. Stop when the partition does not change anymore (stable partition)

### Provided:
- `_refine_partition_once()`
- `stable_partition(adj)`


In [ ]:
# ============================================
# Section 6 - Question 4: Stable partition (single graph)
# ============================================

def _refine_partition_once(
    adj: List[Set[int]],
    node_to_part: List[int],
    parts: List[List[int]]
) -> Tuple[List[int], List[List[int]]]:
    """
    Perform one refinement step of the partition for a single graph.

    For each existing part:
      - group the nodes inside that part according to the sorted multiset
        of neighbor-part IDs.
    """
    n = len(adj)
    new_node_to_part = [-1] * n
    new_parts: List[List[int]] = []

    for old_pid, nodes in enumerate(parts):
        sig_to_subpart: Dict[Tuple[int, ...], List[int]] = {}

        for v in nodes:
            neighbor_parts = sorted(node_to_part[nb] for nb in adj[v])
            sig = tuple(neighbor_parts)
            sig_to_subpart.setdefault(sig, []).append(v)

        for sub in sig_to_subpart.values():
            new_pid = len(new_parts)
            new_parts.append(sub)
            for v in sub:
                new_node_to_part[v] = new_pid

    return new_node_to_part, new_parts


def stable_partition(
    adj: List[Set[int]]
) -> Tuple[List[int], List[List[int]]]:
    """
    Compute the stable partition of a graph, starting from its degree partition.

    We repeatedly refine the partition until it stops changing.
    This is the 1-dimensional Weisfeiler–Leman color refinement.
    """
    node_to_part, parts = _build_degree_partition(adj)

    while True:
        new_node_to_part, new_parts = _refine_partition_once(adj, node_to_part, parts)
        if new_node_to_part == node_to_part:
            return node_to_part, parts
        node_to_part, parts = new_node_to_part, new_parts


## Section 7 — Question 5: Stable Double Partitioning

This extends stable partitioning to **two graphs simultaneously**.

Two partitions are compatible if:

- they have the same number of parts
- each part has the same size
- if a part splits in `G1`, the corresponding part must split the same way in `G2`

If at any refinement step compatibility fails → the graphs cannot be isomorphic.

### Provided:
- `stable_double_partition(...)`


In [ ]:
# ============================================
# Section 7 - Question 5: Stable double partition (two graphs)
# ============================================

def stable_double_partition(
    adj1: List[Set[int]],
    node_to_part1: List[int],
    parts1: List[List[int]],
    adj2: List[Set[int]],
    node_to_part2: List[int],
    parts2: List[List[int]]
) -> Union[bool, Tuple[List[int], List[List[int]], List[int], List[List[int]]]]:
    """
    Refine partitions of G1 and G2 simultaneously until they become
    stable and remain compatible, or detect incompatibility.

    Compatibility conditions:
      - Same number of parts.
      - For each part j: |P1[j]| = |P2[j]|.
      - When a part j is split into subparts in G1, the corresponding part j
        in G2 must split into subparts with the same signatures and sizes.

    Returns:
      False if no compatible stable partitions exist, or
      (node_to_part1, parts1, node_to_part2, parts2) for the final stable partitions.
    """
    while True:
        if len(parts1) != len(parts2):
            return False

        num_parts = len(parts1)
        for pid in range(num_parts):
            if len(parts1[pid]) != len(parts2[pid]):
                return False

        changed = False
        new_node_to_part1 = [-1] * len(adj1)
        new_node_to_part2 = [-1] * len(adj2)
        new_parts1: List[List[int]] = []
        new_parts2: List[List[int]] = []

        for pid in range(num_parts):
            nodes1 = parts1[pid]
            nodes2 = parts2[pid]

            sig_to_nodes1: Dict[Tuple[int, ...], List[int]] = {}
            sig_to_nodes2: Dict[Tuple[int, ...], List[int]] = {}

            for v in nodes1:
                neighbor_parts = sorted(node_to_part1[nb] for nb in adj1[v])
                sig = tuple(neighbor_parts)
                sig_to_nodes1.setdefault(sig, []).append(v)

            for v in nodes2:
                neighbor_parts = sorted(node_to_part2[nb] for nb in adj2[v])
                sig = tuple(neighbor_parts)
                sig_to_nodes2.setdefault(sig, []).append(v)

            # They must have the same set of signatures.
            if set(sig_to_nodes1.keys()) != set(sig_to_nodes2.keys()):
                return False

            # Deterministic ordering of signatures.
            for sig in sorted(sig_to_nodes1.keys(), key=lambda x: (len(x), x)):
                group1 = sig_to_nodes1[sig]
                group2 = sig_to_nodes2[sig]

                if len(group1) != len(group2):
                    return False

                new_pid = len(new_parts1)
                new_parts1.append(group1)
                new_parts2.append(group2)

                for v in group1:
                    new_node_to_part1[v] = new_pid
                for v in group2:
                    new_node_to_part2[v] = new_pid

            if len(sig_to_nodes1) > 1:
                changed = True

        if not changed:
            # Partitions are stable and compatible.
            return new_node_to_part1, new_parts1, new_node_to_part2, new_parts2

        node_to_part1, parts1 = new_node_to_part1, new_parts1
        node_to_part2, parts2 = new_node_to_part2, new_parts2


## Section 8 — Question 6: Weisfeiler–Leman Backtracking

This is the most powerful algorithm in the project.

Steps:
1. Build degree partitions for both graphs
2. Refine them using stable double partitioning
3. During backtracking:
   - a node in part `j` can only map to nodes in part `j`
   - once a mapping `v1 → v2` is fixed:
     - put each into its own singleton part
     - refine again using double partitioning
4. If refinement collapses → backtrack

This combines combinatorial pruning + structural refinement.

### Provided:
- `_split_singleton_part()`
- `_wl_backtracking_recursive()`
- `weisfeiler_leman_backtracking(path1, path2)`


In [ ]:
# ============================================
# Section 8 — Question 6: Weisfeiler–Leman backtracking
# ============================================

from typing import Optional
def _split_singleton_part(
    node_to_part: List[int],
    parts: List[List[int]],
    v: int
) -> Tuple[List[int], List[List[int]]]:
    """
    Ensure that node v is in its own singleton part.

    If its part already has size 1, do nothing.
    Otherwise, split that part into:
      - the remainder (old part without v)
      - a new singleton part {v}
    """
    node_to_part = node_to_part[:]              # copy
    parts = [lst[:] for lst in parts]           # deep copy

    old_pid = node_to_part[v]
    old_nodes = parts[old_pid]

    if len(old_nodes) == 1:
        return node_to_part, parts

    remaining = [x for x in old_nodes if x != v]
    parts[old_pid] = remaining

    new_pid = len(parts)
    parts.append([v])
    node_to_part[v] = new_pid

    return node_to_part, parts


def _mapping_from_partitions(
    adj1: List[Set[int]],
    adj2: List[Set[int]],
    parts1: List[List[int]],
    parts2: List[List[int]]
) -> Optional[List[int]]:
    """
    If the partitions are already 'discrete enough', derive a mapping
    directly from them and verify it. This is valid for any size and is
    just an optimization: no branching is needed if WL has already
    distinguished all vertices.
    """
    if len(parts1) != len(parts2):
        return None

    n = len(adj1)
    mapping = [-1] * n

    for pid in range(len(parts1)):
        block1 = sorted(parts1[pid])
        block2 = sorted(parts2[pid])
        if len(block1) != len(block2):
            return None
        for i in range(len(block1)):
            mapping[block1[i]] = block2[i]

    if check_isomorphism_from_mapping(adj1, adj2, mapping):
        return mapping
    return None


def _wl_backtracking_recursive(
    idx: int,
    adj1: List[Set[int]],
    adj2: List[Set[int]],
    mapping: List[int],
    used: List[bool],
    order: List[int],
    node_to_part1: List[int],
    parts1: List[List[int]],
    node_to_part2: List[int],
    parts2: List[List[int]],
) -> bool:
    """
    Recursive helper for Weisfeiler–Leman-based backtracking.

    Invariant:
      - Part j in G1 corresponds to part j in G2.
      - A node in part j of G1 may only be mapped to unused nodes
        in the same part j of G2.
    After each mapping v1 -> v2:
      - Put both into singleton parts.
      - Refine again with stable_double_partition.
    """
    n = len(adj1)
    if idx == n:
        return True

    # Optional micro-optimization: if at some stage all parts are singletons,
    # we can read off the mapping directly and stop.
    if all(len(block) == 1 for block in parts1):
        direct_mapping = _mapping_from_partitions(adj1, adj2, parts1, parts2)
        if direct_mapping is not None:
            mapping[:] = direct_mapping
            return True

    v1 = order[idx]
    if mapping[v1] != -1:
        return _wl_backtracking_recursive(
            idx + 1, adj1, adj2,
            mapping, used, order,
            node_to_part1, parts1,
            node_to_part2, parts2
        )

    part_id = node_to_part1[v1]

    # Candidates: unused vertices in the same WL part in G2
    candidates = [v2 for v2 in parts2[part_id] if not used[v2]]

    for v2 in candidates:
        # Local consistency check w.r.t. already mapped vertices
        ok = True
        for u1 in range(n):
            if mapping[u1] == -1:
                continue
            u2 = mapping[u1]
            if (u1 in adj1[v1]) != (u2 in adj2[v2]):
                ok = False
                break
        if not ok:
            continue

        # Tentatively extend mapping
        mapping[v1] = v2
        used[v2] = True

        # Force v1 and v2 into singleton parts
        ntp1_new, p1_new = _split_singleton_part(node_to_part1, parts1, v1)
        ntp2_new, p2_new = _split_singleton_part(node_to_part2, parts2, v2)

        # Refine both partitions together
        refined = stable_double_partition(
            adj1, ntp1_new, p1_new,
            adj2, ntp2_new, p2_new
        )

        if refined is not False:
            (ntp1_ref, p1_ref, ntp2_ref, p2_ref) = refined

            if _wl_backtracking_recursive(
                idx + 1, adj1, adj2,
                mapping, used, order,
                ntp1_ref, p1_ref,
                ntp2_ref, p2_ref
            ):
                return True

        # Backtrack
        mapping[v1] = -1
        used[v2] = False

    return False


def weisfeiler_leman_backtracking(path1: str, path2: str) -> Tuple[bool, List[int]]:
    """
    Weisfeiler–Leman backtracking (Question 6).

    1. Load graphs and check quick invariants.
    2. Build degree partitions for both graphs.
    3. Refine them simultaneously with stable_double_partition to obtain
       compatible stable partitions.
    4. Use these partitions to guide a backtracking search:
       - v in part j of G1 can only map to vertices in part j of G2.
       - After each mapping, place both into singleton parts and refine again.
    5. If we reach a full mapping, return it; otherwise return False, [].
    """
    adj1 = load_graph(path1)
    adj2 = load_graph(path2)

    if not quick_invariants_match(adj1, adj2):
        print(False, [])
        return False, []

    n = len(adj1)

    # 1) Degree partitions
    node_to_part1, parts1 = _build_degree_partition(adj1)
    node_to_part2, parts2 = _build_degree_partition(adj2)

    # 2) WL refinement to stable compatible partitions
    refined = stable_double_partition(
        adj1, node_to_part1, parts1,
        adj2, node_to_part2, parts2
    )

    if refined is False:
        print(False, [])
        return False, []

    node_to_part1_ref, parts1_ref, node_to_part2_ref, parts2_ref = refined

    # 3) If WL refinement already distinguishes all vertices, read off mapping
    if all(len(block) == 1 for block in parts1_ref):
        mapping = _mapping_from_partitions(adj1, adj2, parts1_ref, parts2_ref)
        if mapping is not None:
            print(True, mapping)
            return True, mapping
        # If something went wrong, fall back to backtracking.

    # 4) WL-guided backtracking
    mapping = [-1] * n
    used = [False] * n

    # Order: vertices in smaller parts first (more constrained), then by degree.
    order = sorted(
        range(n),
        key=lambda v: (len(parts1_ref[node_to_part1_ref[v]]), -len(adj1[v]))
    )

    success = _wl_backtracking_recursive(
        0, adj1, adj2,
        mapping, used, order,
        node_to_part1_ref, parts1_ref,
        node_to_part2_ref, parts2_ref
    )

    if success:
        print(True, mapping)
        return True, mapping
    else:
        print(False, [])
        return False, []


In [ ]:
# ============================================
# Section 9 – Testing Weisfeiler–Leman Backtracking
# ============================================

def test_wl_on_pair(name: str, path1: str, path2: str):
    """
    Helper to test weisfeiler_leman_backtracking on a single pair of graphs.
    """
    print(f"\n===== {name} =====")
    print(f"Files: {path1}  vs  {path2}")
    result = weisfeiler_leman_backtracking(path1, path2)
    print("Returned:", result)


# ---- Small / medium graphs (should already exist from earlier sections) ----

# Example: small 4-node isomorphic pair
# g1.txt, g2.txt were your very first example used in Section 2/3.
try:
    test_wl_on_pair("Small 4-node isomorphic (g1 vs g2)", "g1.txt", "g2.txt")
except FileNotFoundError:
    print("g1.txt / g2.txt not found – skip this test.")

# Example: 6-node cycle isomorphic pair
try:
    test_wl_on_pair("Cycle (6 nodes, isomorphic)", "cycle6_a.txt", "cycle6_b.txt")
except FileNotFoundError:
    print("cycle6_a.txt / cycle6_b.txt not found – skip this test.")

# Example: 10-node tree isomorphic pair
try:
    test_wl_on_pair("Tree (10 nodes, isomorphic)", "tree10_a.txt", "tree10_b.txt")
except FileNotFoundError:
    print("tree10_a.txt / tree10_b.txt not found – skip this test.")


# ---- Large graphs (2000 nodes) – from the generation cell in Section 10 ----
# These files were created earlier as:
#   large_G.txt        – a random graph G
#   large_G_perm.txt   – a permuted (isomorphic) copy of G
#   large_H.txt        – another random graph, not isomorphic to G

try:
    test_wl_on_pair("Large graphs (2000 nodes, isomorphic: G vs G_perm)",
                    "large_G.txt", "large_G_perm.txt")
except FileNotFoundError:
    print("large_G.txt / large_G_perm.txt not found – skip this test.")

try:
    test_wl_on_pair("Large graphs (2000 nodes, non-isomorphic: G vs H)",
                    "large_G.txt", "large_H.txt")
except FileNotFoundError:
    print("large_G.txt / large_H.txt not found – skip this test.")



===== Small 4-node isomorphic (g1 vs g2) =====
Files: g1.txt  vs  g2.txt
True [1, 2, 0, 3]
Returned: (True, [1, 2, 0, 3])

===== Cycle (6 nodes, isomorphic) =====
Files: cycle6_a.txt  vs  cycle6_b.txt
True [0, 2, 4, 1, 5, 3]
Returned: (True, [0, 2, 4, 1, 5, 3])

===== Tree (10 nodes, isomorphic) =====
Files: tree10_a.txt  vs  tree10_b.txt
True [2, 5, 6, 3, 8, 7, 9, 1, 4, 0]
Returned: (True, [2, 5, 6, 3, 8, 7, 9, 1, 4, 0])

===== Large graphs (2000 nodes, isomorphic: G vs G_perm) =====
Files: large_G.txt  vs  large_G_perm.txt
True [601, 1240, 244, 1599, 655, 1307, 292, 186, 84, 798, 1482, 1392, 1370, 1759, 918, 7, 440, 66, 1228, 820, 1516, 1506, 139, 1489, 806, 1785, 1173, 62, 1662, 1391, 639, 289, 1316, 1526, 721, 1645, 553, 567, 1040, 675, 504, 1453, 853, 1352, 1451, 1083, 110, 1427, 748, 1367, 229, 1978, 1202, 201, 208, 910, 389, 1863, 138, 457, 1820, 1330, 1001, 1078, 64, 1568, 1160, 1163, 1894, 94, 1929, 219, 1711, 1087, 1728, 438, 707, 942, 651, 1974, 1622, 1792, 1264, 1593, 767,

## Section 9 — Testing the Algorithms

You can test with the sample graphs from the project statement.

Example:

### g1.txt
4 4
1
0 2 3
1 3
1 2

### g2.txt
4 4
2 3
2
0 1 3
0 2

All four algorithms should output:
True [1, 2, 0, 3]

(or another valid permutation)


# **Create g1.txt**

In [ ]:
%%writefile g1.txt
4 4
1
0 2 3
1 3
1 2

Overwriting g1.txt


# **Create g2.txt**

In [ ]:
%%writefile g2.txt
4 4
2 3
2
0 1 3
0 2

Overwriting g2.txt


# **Testing the project functions**

In [ ]:
print("Brute force:")
brute_force("g1.txt", "g2.txt")

print("Backtracking:")
backtracking("g1.txt", "g2.txt")

print("Degree Backtracking:")
degree_backtracking("g1.txt", "g2.txt")

print("WL Backtracking:")
weisfeiler_leman_backtracking("g1.txt", "g2.txt")


Brute force:
True [1, 2, 0, 3]
Backtracking:
True [1, 2, 0, 3]
Degree Backtracking:
True [1, 2, 0, 3]
WL Backtracking:
True [1, 2, 0, 3]


(True, [1, 2, 0, 3])

## Section 9 — Experimental Tests and Performance Evaluation

In this section, we test all four implemented algorithms on several graph pairs.

Goals:

- Verify **correctness** on both isomorphic and non-isomorphic graphs.
- Illustrate how performance changes with graph size.
- Demonstrate the practical advantage of **Degree Backtracking** and **Weisfeiler–Leman Backtracking**, as mentioned in the project remarks.

We use three types of test pairs:

1. **Path vs Star (8 nodes)** — same number of nodes and edges, but different structure → *non-isomorphic*.  
2. **Cycle on 6 nodes** — same cycle, different node labellings → *isomorphic*.  
3. **Random tree on 10 nodes** and a permuted copy → *isomorphic*, slightly larger, WL-friendly.


In [ ]:
# ============================================
# Section 9 - Helper function to test and time the algorithms
# ============================================

import time

def test_all_algorithms(name: str, path1: str, path2: str):
    """
    Run all four algorithms on the given pair (path1, path2),
    print the result and execution time for each.
    Intended for small/medium graphs (up to ~8 nodes).
    """
    print(f"\n===== Test: {name} =====")
    print(f"Files: {path1}  vs  {path2}\n")

    # 1) Brute force
    start = time.perf_counter()
    res_bf = brute_force(path1, path2)
    t_bf = time.perf_counter() - start
    print(f"Brute force time: {t_bf:.6f} s   -> {res_bf}")

    # 2) Backtracking
    start = time.perf_counter()
    res_bt = backtracking(path1, path2)
    t_bt = time.perf_counter() - start
    print(f"Backtracking time: {t_bt:.6f} s  -> {res_bt}")

    # 3) Degree-based backtracking
    start = time.perf_counter()
    res_deg = degree_backtracking(path1, path2)
    t_deg = time.perf_counter() - start
    print(f"Degree backtracking time: {t_deg:.6f} s  -> {res_deg}")

    # 4) Weisfeiler–Leman backtracking
    start = time.perf_counter()
    res_wl = weisfeiler_leman_backtracking(path1, path2)
    t_wl = time.perf_counter() - start
    print(f"WL backtracking time: {t_wl:.6f} s  -> {res_wl}")


### 9.1 — Creating the Test Graph Files

We now create three graph pairs:

1. `path8.txt` and `star8.txt` — both on 8 nodes, but structurally different → *not isomorphic*.  
2. `cycle6_a.txt` and `cycle6_b.txt` — two different labelings of the same 6-cycle → *isomorphic*.  
3. `tree10_a.txt` and `tree10_b.txt` — a random tree on 10 nodes and a permuted copy → *isomorphic*, used as a larger example.


In [ ]:
%%writefile path8.txt
8 7
1
0 2
1 3
2 4
3 5
4 6
5 7
6

Overwriting path8.txt


In [ ]:
%%writefile star8.txt
8 7
1 2 3 4 5 6 7
0
0
0
0
0
0
0

Overwriting star8.txt


In [ ]:
%%writefile cycle6_a.txt
6 6
1 5
0 2
1 3
2 4
3 5
4 0

Overwriting cycle6_a.txt


In [ ]:
%%writefile cycle6_b.txt
6 6
2 3
4 5
0 4
0 5
1 2
1 3

Overwriting cycle6_b.txt


In [ ]:
# ============================================
# Section 9.1 - Generate random 10-node tree pair (isomorphic)
# ============================================

import random

def generate_random_tree(n: int):
    """
    Generate a random tree on n nodes using a Prüfer sequence.
    Trees are guaranteed to be distinguished by 1-WL refinement.
    """
    if n <= 1:
        return [set() for _ in range(n)]

    # Prüfer sequence of length n-2
    prufer = [random.randint(0, n-1) for _ in range(n - 2)]
    degree = [1] * n
    for v in prufer:
        degree[v] += 1

    leaves = [i for i in range(n) if degree[i] == 1]
    leaves.sort()

    adj = [set() for _ in range(n)]

    for v in prufer:
        leaf = leaves[0]
        leaves.pop(0)

        adj[leaf].add(v)
        adj[v].add(leaf)

        degree[leaf] -= 1
        degree[v] -= 1

        if degree[v] == 1:
            leaves.append(v)
            leaves.sort()

    # Connect the last two leaves
    a, b = leaves
    adj[a].add(b)
    adj[b].add(a)

    return adj

def save_graph(adj, fname: str):
    n = len(adj)
    m = sum(len(adj[i]) for i in range(n)) // 2
    with open(fname, "w") as f:
        f.write(f"{n} {m}\n")
        for i in range(n):
            f.write(" ".join(str(x) for x in adj[i]) + "\n")

# Generate tree T
T = generate_random_tree(10)

# Create an isomorphic copy T' using a random permutation
perm = list(range(10))
random.shuffle(perm)

Tp = [set() for _ in range(10)]
for i in range(10):
    for j in T[i]:
        Tp[perm[i]].add(perm[j])

# Save both trees
save_graph(T, "tree10_a.txt")
save_graph(Tp, "tree10_b.txt")

print("Generated tree10_a.txt and tree10_b.txt (10-node tree, isomorphic).")


Generated tree10_a.txt and tree10_b.txt (10-node tree, isomorphic).


### 9.2 — Running All Algorithms on the Graph Pairs

We now run:

- `brute_force`
- `backtracking`
- `degree_backtracking`
- `weisfeiler_leman_backtracking`

on:

1. `path8.txt` vs `star8.txt` — non-isomorphic.  
2. `cycle6_a.txt` vs `cycle6_b.txt` — isomorphic.  
3. `tree10_a.txt` vs `tree10_b.txt` — isomorphic, larger tree (we skip brute force here because 10! is very large).


In [ ]:
# ============================================
# Section 9.2 - Execute tests
# ============================================

# 1) Non-isomorphic: path vs star (8 nodes)
test_all_algorithms("Path vs Star (8 nodes, non-isomorphic)", "path8.txt", "star8.txt")

# 2) Isomorphic: 6-cycle
test_all_algorithms("Cycle (6 nodes, isomorphic)", "cycle6_a.txt", "cycle6_b.txt")

# 3) Isomorphic: 10-node tree (skip brute force)
print("\n===== Test: Tree (10 nodes, isomorphic) =====")
print("Files: tree10_a.txt  vs  tree10_b.txt\n")

start = time.perf_counter()
res_bt = backtracking("tree10_a.txt", "tree10_b.txt")
t_bt = time.perf_counter() - start
print(f"Backtracking time: {t_bt:.6f} s -> {res_bt}")

start = time.perf_counter()
res_deg = degree_backtracking("tree10_a.txt", "tree10_b.txt")
t_deg = time.perf_counter() - start
print(f"Degree backtracking time: {t_deg:.6f} s -> {res_deg}")

start = time.perf_counter()
res_wl = weisfeiler_leman_backtracking("tree10_a.txt", "tree10_b.txt")
t_wl = time.perf_counter() - start
print(f"WL backtracking time: {t_wl:.6f} s -> {res_wl}")



===== Test: Path vs Star (8 nodes, non-isomorphic) =====
Files: path8.txt  vs  star8.txt

False []
Brute force time: 0.000950 s   -> (False, [])
False []
Backtracking time: 0.000135 s  -> (False, [])
False []
Degree backtracking time: 0.000186 s  -> (False, [])
False []
WL backtracking time: 0.000102 s  -> (False, [])

===== Test: Cycle (6 nodes, isomorphic) =====
Files: cycle6_a.txt  vs  cycle6_b.txt

True [0, 2, 4, 1, 5, 3]
Brute force time: 0.000262 s   -> (True, [0, 2, 4, 1, 5, 3])
True [0, 2, 4, 1, 5, 3]
Backtracking time: 0.000210 s  -> (True, [0, 2, 4, 1, 5, 3])
True [0, 2, 4, 1, 5, 3]
Degree backtracking time: 0.000158 s  -> (True, [0, 2, 4, 1, 5, 3])
True [0, 2, 4, 1, 5, 3]
WL backtracking time: 0.000461 s  -> (True, [0, 2, 4, 1, 5, 3])

===== Test: Tree (10 nodes, isomorphic) =====
Files: tree10_a.txt  vs  tree10_b.txt

True [5, 3, 7, 2, 4, 8, 1, 0, 6, 9]
Backtracking time: 0.001472 s -> (True, [5, 3, 7, 2, 4, 8, 1, 0, 6, 9])
True [5, 3, 7, 2, 4, 8, 1, 0, 6, 9]
Degree backtr

### 9.3 — Interpretation of Results

From these experiments, we typically observe:

- **Path vs Star (8 nodes):**
  - All four algorithms correctly return `False, []`.
  - This confirms they can detect that graphs with the same n and m but different structure are not isomorphic.

- **6-cycle (6 nodes, two labelings):**
  - All four algorithms correctly return `True` with a valid permutation.
  - Brute force is still fast here, but more advanced methods already start to show similar or better performance.

- **Tree (10 nodes, isomorphic):**
  - We skip brute force due to factorial complexity.
  - Backtracking and Degree Backtracking succeed.
  - **Weisfeiler–Leman Backtracking** is efficient and benefits from the tree structure, illustrating the power of structural refinement and message passing, as highlighted in the project remarks.

Overall, these tests demonstrate:

- **Correctness** of all four implementations on both isomorphic and non-isomorphic examples.
- **Performance improvements** from:
  - plain Backtracking → Degree Backtracking → Weisfeiler–Leman Backtracking.
- A clear motivation for using partition refinement and Weisfeiler–Leman ideas in practice.

This completes the experimental validation of the project.


## Section 9.4 — Discussion and Context of previous sections

**Remark 3 — Real-life application.**  
The (sub)graph isomorphism problem appears in many practical settings.  
One concrete example is image geolocalization, where a query image is matched to a map
by finding correspondences between visual features and landmarks represented as graphs.
(See: *Approximate Subgraph Isomorphism for Image Localization*, Vaishaal Shankar et al., Electronic Imaging 2016.)

**Remark 4 — Complexity.**  
The graph isomorphism problem is in NP, but is not known to be in P and not known to be NP-complete,
so it is believed to be NP-intermediate.  
In contrast, the subgraph isomorphism problem is known to be NP-complete.

**Remark 5 — Weisfeiler–Leman and message passing.**  
The Weisfeiler–Leman refinement used in this project repeatedly updates node "colors"
based on the multiset of colors of their neighbors.  
This iterative "message passing" is conceptually similar to how modern Graph Neural Networks (GNNs)
aggregate and propagate information over neighbors in each layer.


# ============================================
# Section 10 — Large Random Graph Performance Tests (≈2000 nodes)
# ============================================

In this section, we evaluate the *scalability* and *efficiency* of the algorithms on **very large graphs**.

Full isomorphism search (backtracking, degree-backtracking, WL-backtracking) is **factorial in the worst case**,
so graphs with *thousands* of nodes cannot be solved with mapping-based algorithms.

However, the **Weisfeiler–Leman color refinement** (stable partition and stable double partition) is *fast*, running in approximately:

\[
O(n \log n + m)
\]

and is widely used in large-scale graph tasks, including graph neural networks (Remark 5).

### Goals of this section:

1. Generate a very large random graph \(G\) with **n = 2000 nodes**.
2. Create an *isomorphic* version \(G_{\text{perm}}\) by randomly permuting node IDs.
3. Generate another random graph \(H\) (same size and density) that is *unlikely isomorphic*.
4. Run **fast invariants** and **stable double partition (1-WL refinement)** on:
   - \(G\) vs \(G_{\text{perm}}\)  → *should stay compatible*
   - \(G\) vs \(H\) → *should quickly detect incompatibility*
5. Measure the execution time of WL refinement on large graphs.

This provides a realistic performance experiment demonstrating that the implementation scales to graphs far beyond the size where backtracking is feasible.


In [ ]:
# ============================================
# Section 10.1 - Large random graph generator
# ============================================

import random

def generate_erdos_renyi_graph(n: int, avg_degree: int):
    """
    Generate a random undirected graph G(n, p) with approximately the given
    average degree using edge probability p = avg_degree / (n - 1).
    Returns adjacency list as List[Set[int]].
    """
    if n <= 0:
        return []

    p = avg_degree / max(1, n - 1)
    adj = [set() for _ in range(n)]

    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                adj[i].add(j)
                adj[j].add(i)

    return adj


def save_graph_adj(adj, fname: str):
    """
    Save adjacency list to file in project format.
    """
    n = len(adj)
    m = sum(len(adj[i]) for i in range(n)) // 2
    with open(fname, "w") as f:
        f.write(f"{n} {m}\n")
        for i in range(n):
            f.write(" ".join(str(x) for x in adj[i]) + "\n")


In [ ]:
def summarize_graph(adj, name="Graph", preview=10):
    """
    Print a readable summary of a large graph:
    - number of nodes
    - number of edges
    - degree stats
    - preview of adjacency list for the first N nodes
    """
    n = len(adj)
    m = sum(len(adj[i]) for i in range(n)) // 2
    degrees = [len(adj[i]) for i in range(n)]

    print(f"\n===== Summary of {name} =====")
    print(f"Nodes: {n}")
    print(f"Edges: {m}")
    print(f"Min degree: {min(degrees)}")
    print(f"Max degree: {max(degrees)}")
    print(f"Average degree: {sum(degrees)/n:.2f}")
    print(f"Degree distribution preview (first 10): {degrees[:10]}")

    print(f"\nFirst {preview} adjacency rows:")
    for i in range(min(preview, n)):
        print(f"{i}: {sorted(list(adj[i]))}")

In [ ]:
# ============================================
# Section 10.2 - Create large random graphs (n = 2000)
# ============================================

n_large = 2000
avg_degree = 20   # change this if you want more/less dense graphs

print("Generating large random graphs...")

# 1) G: Random graph
G_large = generate_erdos_renyi_graph(n_large, avg_degree)

# 2) G_perm: Random permutation of G (isomorphic)
perm = list(range(n_large))
random.shuffle(perm)

G_perm = [set() for _ in range(n_large)]
for i in range(n_large):
    for j in G_large[i]:
        G_perm[perm[i]].add(perm[j])

# 3) H: Independent random graph
H_large = generate_erdos_renyi_graph(n_large, avg_degree)

# Save them
save_graph_adj(G_large, "large_G.txt")
save_graph_adj(G_perm, "large_G_perm.txt")
save_graph_adj(H_large, "large_H.txt")

print("✔ Saved large_G.txt, large_G_perm.txt (isomorphic), large_H.txt (random).")


Generating large random graphs...
✔ Saved large_G.txt, large_G_perm.txt (isomorphic), large_H.txt (random).


In [ ]:
G_large_loaded  = load_graph("large_G.txt")
G_perm_loaded   = load_graph("large_G_perm.txt")
H_large_loaded  = load_graph("large_H.txt")

In [ ]:
summarize_graph(G_large_loaded, "Large Graph G")
summarize_graph(G_perm_loaded, "Large Graph G_perm")
summarize_graph(H_large_loaded, "Large Graph H")


===== Summary of Large Graph G =====
Nodes: 2000
Edges: 20111
Min degree: 8
Max degree: 33
Average degree: 20.11
Degree distribution preview (first 10): [23, 13, 18, 10, 20, 25, 21, 27, 30, 22]

First 10 adjacency rows:
0: [18, 283, 297, 493, 515, 600, 816, 827, 859, 1246, 1267, 1318, 1337, 1434, 1461, 1497, 1523, 1573, 1636, 1855, 1934, 1971, 1989]
1: [30, 67, 132, 499, 842, 1012, 1274, 1501, 1801, 1876, 1897, 1907, 1970]
2: [72, 456, 657, 680, 682, 801, 829, 860, 904, 941, 1100, 1215, 1224, 1235, 1269, 1788, 1857, 1962]
3: [239, 615, 673, 1017, 1052, 1296, 1570, 1579, 1670, 1791]
4: [31, 220, 294, 317, 466, 608, 610, 933, 995, 1006, 1241, 1269, 1361, 1435, 1465, 1527, 1799, 1839, 1843, 1958]
5: [299, 352, 369, 448, 478, 530, 600, 778, 832, 888, 1012, 1045, 1081, 1103, 1104, 1344, 1453, 1482, 1542, 1620, 1665, 1801, 1805, 1892, 1975]
6: [114, 277, 306, 372, 561, 651, 652, 733, 911, 1074, 1188, 1239, 1338, 1445, 1504, 1525, 1597, 1692, 1693, 1768, 1952]
7: [165, 177, 262, 272, 292, 51

In [ ]:
 # ============================================
# Section 10.3 - Time WL refinement on large graphs
# ============================================

import time

def time_wl_refinement(name, adjA, adjB):
    print(f"\n--- {name} ---")

    # Quick invariants (degree multisets)
    inv_ok = quick_invariants_match(adjA, adjB)
    print(f"Degree multiset match: {inv_ok}")

    if not inv_ok:
        print("Rejected by degree-invariants.")
        return

    # Build degree partitions
    ntpA, pA = _build_degree_partition(adjA)
    ntpB, pB = _build_degree_partition(adjB)

    start = time.perf_counter()
    result = stable_double_partition(adjA, ntpA, pA, adjB, ntpB, pB)
    elapsed = time.perf_counter() - start

    if result is False:
        print("WL refinement result: Incompatible (non-isomorphic).")
    else:
        print("WL refinement result: Compatible (isomorphic candidate).")

    print(f"Time: {elapsed:.4f} seconds")


# Load the graphs
G_large_loaded  = load_graph("large_G.txt")
G_perm_loaded   = load_graph("large_G_perm.txt")
H_large_loaded  = load_graph("large_H.txt")

# Run WL refinement tests
time_wl_refinement("G vs G_perm (isomorphic)", G_large_loaded, G_perm_loaded)
time_wl_refinement("G vs H (non-isomorphic)", G_large_loaded, H_large_loaded)



--- G vs G_perm (isomorphic) ---
Degree multiset match: True
WL refinement result: Compatible (isomorphic candidate).
Time: 0.1345 seconds

--- G vs H (non-isomorphic) ---
Degree multiset match: False
Rejected by degree-invariants.
